# Home Cage Elo Score Calculation

## Importing other Python Libraries/Modules

In [1]:
import re
import os
import sys
from collections import defaultdict
import glob
import warnings
from datetime import datetime
from datetime import timedelta
import ast

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xlrd

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
from eloscore import eloscore

In [5]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

## Summary

- Experiment
    - Observing fighting behavior between mice in their home cage in different settings. Mice are categorized as winner and loser based on if they initiated the behavior or if they are recieving it. Recording sessions are usually from 30 minutes to a hour with cages of 2 to 6 mice.
- Data
    - Excel spreadsheet of recorded fighting behavior. The relevant columns are those of the "Date" of the recording, the "winner" of the interaction, and the "loser" of the interaction. Each row will be for one interaction between two mice.
    - There is a cage for each sheet of the spreadsheet
    - For each recording session, we will assume that the date will be specified for that first row. We will use this to seperate all the rows into seperate sessions.
- Purpose of this Jupyter Notebook
    - To calculate the Elo Score of each mice after each interaction. The mice start off with an Elo score of 1000. Elo scores are calculated with the formula from here: https://www.omnicalculator.com/sports/elo . Then we will plot the change of elo score across all interactions. With the number of interactions on the X-Axis and the current Elo score on the Y. There will be a line for mice.

## Getting the file name of the raw data

- Default input folder and keyword to search the files for 
    - **NOTE**: This should not be changed unless there is a consistent change with the file naming convention

In [6]:
input_folder = "./data"

In [7]:
# Accomodates for both capitalization of the file names
raw_data_file_keyword = '*[hH]ome*'

- Asking the user what the path to the recording files are, with the option of using wildcards

In [8]:
raw_data_glob_pattern = input("""Type out the path(address) of the raw behavioral recording excel sheets.
Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

Globbing can also be used if you want to search with a wild card(Capitalization matters). 
i.e. './data/*Home*' will be able to find './data/Homecage_observations.xlsx'

NOTE: If left blank, the path will automatically be guessed off of the default settings 
""")

Type out the path(address) of the raw behavioral recording excel sheets.
Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

Globbing can also be used if you want to search with a wild card(Capitalization matters). 
i.e. './data/*Home*' will be able to find './data/Homecage_observations.xlsx'

NOTE: If left blank, the path will automatically be guessed off of the default settings 
./data/*Home*


- By default, this will search for files that are in the `./data` folder (in the folder that this notebook is in) that have `home` in the file name

In [9]:
# Using the user inputted path/pattern
if raw_data_glob_pattern.strip():
    raw_data_files_list = glob.glob(raw_data_glob_pattern.strip())
    if raw_data_files_list:
        raw_data_file_path = raw_data_files_list[0]
    else:
        raise ValueError("No files were found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))

# Using the default pattern
else:
    # Searching for matching files with recursion
    if glob.glob(os.path.join(input_folder + "**", raw_data_file_keyword)):
        raw_data_glob_pattern = os.path.join(input_folder + "**", raw_data_file_keyword)
        raw_data_files_list = glob.glob(raw_data_glob_pattern)
    # Searching for matching files only in the folder
    elif glob.glob(os.path.join(input_folder, raw_data_file_keyword)):
        raw_data_glob_pattern = glob.glob(os.path.join(input_folder, raw_data_file_keyword))
        raw_data_files_list = glob.glob(raw_data_glob_pattern)
    else:
        raise ValueError("No files were found in {} that had the keyword {} in it".format(input_folder, raw_data_file_keyword))


- Checking to see if only one file is specified

In [10]:
if len(raw_data_files_list) >= 2:
    raise ValueError("More than one file was found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))
else:
    raw_data_file_path = raw_data_files_list[0]

In [11]:
raw_data_file_path

'./data/Home_Cage_Behaviors_Data.xlsx'

## Getting a list of all the sheets

In [12]:
xls = pd.ExcelFile(raw_data_file_path)
raw_data_sheet_names = xls.sheet_names

In [13]:
raw_data_sheet_names

['Legend and Scoring', 'CAGE1', 'CAGE2', 'CAGE3', 'CAGE4', 'CAGE5', 'CAGE6']

In [14]:
inputted_sheet_names_string = input("""Type out the name of the sheets that you want to be processed. 
Each name must be put in quotes and seperated by a comma(,). i.e. "CAGE3", "CAGE4"

Alternatively, you can use the index of the list of names above. 
Remember, that Python is zero indexed so the first item will be have the 0 index, second the 1 index, and so on. 
i.e. 1, 2

NOTE: If left blank, all sheets will be used
""")

Type out the name of the sheets that you want to be processed. 
Each name must be put in quotes and seperated by a comma(,). i.e. "CAGE3", "CAGE4"

Alternatively, you can use the index of the list of names above. 
Remember, that Python is zero indexed so the first item will be have the 0 index, second the 1 index, and so on. 
i.e. 1, 2

NOTE: If left blank, all sheets will be used
'CAGE1', 'CAGE2', 'CAGE3', 'CAGE4', 'CAGE5', 'CAGE6'


In [15]:
# Making a list out of the string of inputted sheet names
if inputted_sheet_names_string:
    inputted_sheet_names_string = "[" + inputted_sheet_names_string + "]"
    inputted_sheet_names_list = ast.literal_eval(inputted_sheet_names_string)
# Using all the sheet names if no sheet name is specified
else:
    inputted_sheet_names_list =  raw_data_sheet_names

In [16]:
inputted_sheet_names_list

['CAGE1', 'CAGE2', 'CAGE3', 'CAGE4', 'CAGE5', 'CAGE6']

- Converting all the numbers into the sheet name that the index of the number corresponds to

In [17]:
for index, sheet in enumerate(inputted_sheet_names_list):
    if str(sheet).isdigit():
        inputted_sheet_names_list[index] =  raw_data_sheet_names[sheet]

In [18]:
inputted_sheet_names_list

['CAGE1', 'CAGE2', 'CAGE3', 'CAGE4', 'CAGE5', 'CAGE6']

- Checking to see if all the sheets are in the excel file

In [19]:
if not set(inputted_sheet_names_list).issubset(raw_data_sheet_names):
    not_included_sheet_names = set(inputted_sheet_names_list) - set(raw_data_sheet_names)
    raise ValueError("All the listed sheet names are not in {}".format(not_included_sheet_names))

## Finding the row for the header

- Showing which columns would be used 

In [20]:
sheet_name_to_header_row = defaultdict(dict)
for sheet in inputted_sheet_names_list:
    print("\nCurrent Sheet Name: {}".format(sheet))
    per_sheet_dataframe = pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0)
    # Showing the columns that are chosen with the header being the 0th row
    print("Columns Names: {}".format(per_sheet_dataframe.columns))
    # Show the dataframe that would be created with the header being the 0th row
    print("First few rows of this dataframe:")
    print(pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0).head())
    # Allowing the user the choose the row number for the header
    header_row = int(input("""Type the row number to be used as the header
    (AKA the row with the column name that you want to use.)
    If you want to keep the column names that were displayed, type 0.
    If you want to use a different row, then type the corresponding number. 
    
    The rows displayed in this cell are dataframes created from Pandas. 
    To use the row with the 0 index for column names, type 1. 
    For the row with the 1 index, it will be 2 and so on.
    
    If you are looking at the original spread sheet, remember that Python is zero indexed. 
    So the first row will be 0, second will be 1, and so on. 
    i.e. 1
    
    NOTE: If left blank, the original row that was used will be used.
    """).strip())
    if not header_row:
        header_row = 0
    
    column_names = "".join([str(col) for col in pd.read_excel(raw_data_file_path, sheet_name=sheet, header=header_row).columns])
    if "Unnamed" in column_names:
        raise ValueError("""Not all the cells in the chosen row are filled in.
                         Please choose a row that has the name of the columns
                         
                         The values in this row are: {}""".format(column_names))
    else:
        sheet_name_to_header_row[sheet]["header_row"] = header_row


Current Sheet Name: CAGE1
Columns Names: Index(['Unnamed: 0', 'any details on method ', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'who gets the point and who loses one (only Fights and Chases get points). The winner gets 1 point and loser -1 ',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')
First few rows of this dataframe:
  Unnamed: 0                          any details on method   \
0   observer                                           notes   
1    Isabel    9:27-9:57am, C57 lots of grooming and rearing   
2        NaN                                             NaN   
3        NaN                                             NaN   
4        NaN                                             NaN   

               Unnamed: 2           Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0  length of observations                 date     cage #     winner   
1                   30min  2022-06-29 00:00:00          1    

Type the row number to be used as the header
    (AKA the row with the column name that you want to use.)
    If you want to keep the column names that were displayed, type 0.
    If you want to use a different row, then type the corresponding number. 
    
    The rows displayed in this cell are dataframes created from Pandas. 
    To use the row with the 0 index for column names, type 1. 
    For the row with the 1 index, it will be 2 and so on.
    
    If you are looking at the original spread sheet, remember that Python is zero indexed. 
    So the first row will be 0, second will be 1, and so on. 
    i.e. 1
    
    NOTE: If left blank, the original row that was used will be used.
    1

Current Sheet Name: CAGE4
Columns Names: Index(['Unnamed: 0',
       'any details on method (e.g. right after tube test or isolation ,etc)',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'who gets the point and who loses one (only Fights and Chases get point

Type the row number to be used as the header
    (AKA the row with the column name that you want to use.)
    If you want to keep the column names that were displayed, type 0.
    If you want to use a different row, then type the corresponding number. 
    
    The rows displayed in this cell are dataframes created from Pandas. 
    To use the row with the 0 index for column names, type 1. 
    For the row with the 1 index, it will be 2 and so on.
    
    If you are looking at the original spread sheet, remember that Python is zero indexed. 
    So the first row will be 0, second will be 1, and so on. 
    i.e. 1
    
    NOTE: If left blank, the original row that was used will be used.
    1


In [21]:
sheet_name_to_header_row

defaultdict(dict,
            {'CAGE1': {'header_row': 1},
             'CAGE2': {'header_row': 1},
             'CAGE3': {'header_row': 1},
             'CAGE4': {'header_row': 1},
             'CAGE5': {'header_row': 1},
             'CAGE6': {'header_row': 1}})

## Reading in all the spreadsheets

In [22]:
for key, value in sheet_name_to_header_row.items():
    sheet_name_to_header_row[key]["original_behavior_recording_dataframe"] = pd.read_excel(raw_data_file_path, sheet_name=key, header=value["header_row"])

In [23]:
sheet_name_to_header_row[key]["original_behavior_recording_dataframe"].head()

,observer,notes,length of observations,date,cage #,winner,loser,action,1,2,3,4
0,Jocelyn,AM,30 min,2022-06-29,6.0,3,1,fs,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaT,NaN,4,2,fs,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaT,NaN,4,3,fs,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaT,NaN,2,1,fs,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaT,NaN,4,3,as,NaN,NaN,NaN,NaN


## Standarizing the Columns

- Making all the column names lower case and removing any extra spaces in the beginning and at the end

In [24]:
for key, value in sheet_name_to_header_row.items():
    column_name_to_standarized = defaultdict(dict)
    for col in sheet_name_to_header_row[key]["original_behavior_recording_dataframe"]:
        column_name_to_standarized[col] = str(col).lower().strip()
    sheet_name_to_header_row[key]["column_name_to_standarized"] = column_name_to_standarized

In [25]:
sheet_name_to_header_row[key]["column_name_to_standarized"]

defaultdict(dict,
            {'observer': 'observer',
             'notes ': 'notes',
             'length of observations': 'length of observations',
             'date': 'date',
             'cage #': 'cage #',
             'winner': 'winner',
             'loser': 'loser',
             'action': 'action',
             1: '1',
             2: '2',
             3: '3',
             4: '4'})

In [27]:
for key, value in sheet_name_to_header_row.items():
    sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"] = sheet_name_to_header_row[key]["original_behavior_recording_dataframe"].rename(columns=sheet_name_to_header_row[key]["column_name_to_standarized"])

In [28]:
sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].head()

,observer,notes,length of observations,date,cage #,winner,loser,action,1,2,3,4
0,Jocelyn,AM,30 min,2022-06-29,6.0,3,1,fs,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaT,NaN,4,2,fs,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaT,NaN,4,3,fs,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaT,NaN,2,1,fs,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaT,NaN,4,3,as,NaN,NaN,NaN,NaN


## Adding the session number

- Session number will be used for plotting

In [30]:
for key, value in sheet_name_to_header_row.items():
    # Allowing the user to choose the row number for the header
    session_divider_column = input("""Type the name of the column to be used to divide the rows into sessions. 
    Pick a column that has a value in the cell only with the first row of each session.
    This is usually the "date" column.

    If left blank, the session number will not be added to the spreadsheets and plots will not be divided into sessions.

    The available columns are: {}

    NOTE: If left blank, "date" will be used as the column
    """.format(sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].columns))

    sheet_name_to_header_row[key]["session_divider_column"] = session_divider_column.lower().strip('"').strip("'").strip()


    if not sheet_name_to_header_row[key]["session_divider_column"]:
        sheet_name_to_header_row[key]["session_divider_column"] = "date"

    if sheet_name_to_header_row[key]["session_divider_column"] not in sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].columns:
        raise ValueError("{} is not a column in {}".format(sheet_name_to_header_row[key]["session_divider_column"], home_cage_observations_df.columns)) 

Type the name of the column to be used to divide the rows into sessions. 
    Pick a column that has a value in the cell only with the first row of each session.
    This is usually the "date" column.

    If left blank, the session number will not be added to the spreadsheets and plots will not be divided into sessions.

    The available columns are: Index(['observer', 'notes', 'length of observations', 'date', 'cage #',
       'winner', 'loser', 'action', '1', '2', '3', '4'],
      dtype='object')

    NOTE: If left blank, "date" will be used as the column
    
Type the name of the column to be used to divide the rows into sessions. 
    Pick a column that has a value in the cell only with the first row of each session.
    This is usually the "date" column.

    If left blank, the session number will not be added to the spreadsheets and plots will not be divided into sessions.

    The available columns are: Index(['observer', 'notes', 'length of observations', 'date', 'cage #',
  

- Getting the indexes of where each new session starts

In [31]:
for key, value in sheet_name_to_header_row.items():
    sheet_name_to_header_row[key]["session_indexes"] = sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].dropna(axis=0, subset=sheet_name_to_header_row[key]["session_divider_column"]).index
    
    sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"] = eloscore.add_session_number_column(sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"], sheet_name_to_header_row[key]["session_indexes"], session_number_column='session_number')
    sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"] = sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].fillna(method='ffill')

In [32]:
sheet_name_to_header_row[key]["session_indexes"]

Int64Index([0, 65, 222, 243, 299, 335, 393, 425, 435, 496, 532, 545, 553, 567,
            572, 586],
           dtype='int64')

In [33]:
sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].head()

,observer,notes,length of observations,date,cage #,winner,loser,action,1,2,3,4,session_number
0,Jocelyn,AM,30 min,2022-06-29,6.0,3,1,fs,NaN,NaN,NaN,NaN,1.0
1,Jocelyn,AM,30 min,2022-06-29,6.0,4,2,fs,NaN,NaN,NaN,NaN,1.0
2,Jocelyn,AM,30 min,2022-06-29,6.0,4,3,fs,NaN,NaN,NaN,NaN,1.0
3,Jocelyn,AM,30 min,2022-06-29,6.0,2,1,fs,NaN,NaN,NaN,NaN,1.0
4,Jocelyn,AM,30 min,2022-06-29,6.0,4,3,as,NaN,NaN,NaN,NaN,1.0


In [34]:
sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].tail()

,observer,notes,length of observations,date,cage #,winner,loser,action,1,2,3,4,session_number
587,Catalina,"while being food restricted, before being fed",30 min,2022-08-09,6.0,1,4,f,1.0,-1.0,-1.0,-1.0,16.0
588,Catalina,"while being food restricted, before being fed",30 min,2022-08-09,6.0,1,4,f,1.0,-1.0,-1.0,-1.0,16.0
589,Catalina,"while being food restricted, before being fed",30 min,2022-08-09,6.0,1,2,c,1.0,1.0,-1.0,-1.0,16.0
590,Catalina,"while being food restricted, before being fed",30 min,2022-08-09,6.0,1,2,c,1.0,1.0,-1.0,-1.0,16.0
591,Catalina,"while being food restricted, before being fed",30 min,2022-08-09,6.0,1,4,f,1.0,-1.0,-1.0,-1.0,16.0


- Adding a column for the session number and then filling all the `NaNs`

## Dropping the rows with only Chase and Fights

In [35]:
for key, value in sheet_name_to_header_row.items():
    # Allowing the user the choose the row number for the header
    sheet_name_to_header_row[key]["action_column"] = input("""Type the name of the column that has the action recorded.

    The available columns are: {}

    If left blank, the column name will assumed to be "action"
    """.format(sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].columns))

    if sheet_name_to_header_row[key]["action_column"]:
        sheet_name_to_header_row[key]["action_column"] = sheet_name_to_header_row[key]["action_column"].lower().strip()
    else: 
        sheet_name_to_header_row[key]["action_column"] = "action"
        
    # Allowing the user the choose the row number for the header
    sheet_name_to_header_row[key]["elo_counted_actions_string"] = input("""Type the actions that will be counted towards ELO Score points.
    All the available actions: {}
    Each action must be put in quotes and seperated by a comma(,). i.e. "f", "c"

    NOTE: If left blank, the ELO Score counted actions will assumed to be "f", "c"
    """.format(sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"][sheet_name_to_header_row[key]["action_column"]].unique()))
    
    # Making a list out of the string of inputted sheet names
    if sheet_name_to_header_row[key]["elo_counted_actions_string"]:
        sheet_name_to_header_row[key]["elo_counted_actions_string"] = "[" + sheet_name_to_header_row[key]["elo_counted_actions_string"] + "]"
        sheet_name_to_header_row[key]["elo_counted_actions_list"] = ast.literal_eval(sheet_name_to_header_row[key]["elo_counted_actions_string"])
        sheet_name_to_header_row[key]["elo_counted_actions_list"] = [action.strip() for action in sheet_name_to_header_row[key]["elo_counted_actions_list"]]
    # Using all the sheet names if no sheet name is specified
    else:
        sheet_name_to_header_row[key]["elo_counted_actions_list"] =  ["f", "c"]
    
    
    sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"] = sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"][sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"][sheet_name_to_header_row[key]["action_column"]].isin(sheet_name_to_header_row[key]["elo_counted_actions_list"])]
    
    sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"]['session_number_difference'] = sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"]['session_number'].diff()

Type the name of the column that has the action recorded.

    The available columns are: Index(['observer', 'notes', 'length of observations', 'date', 'cage #',
       'winner', 'loser', 'action', '1', '2', '3', '4', 'session_number'],
      dtype='object')

    If left blank, the column name will assumed to be "action"
    
Type the actions that will be counted towards ELO Score points.
    All the available actions: ['fs' 'ag' 'as' 'fs ' 'c' 'f' 'ag ']
    Each action must be put in quotes and seperated by a comma(,). i.e. "f", "c"

    NOTE: If left blank, the ELO Score counted actions will assumed to be "f", "c"
    
Type the name of the column that has the action recorded.

    The available columns are: Index(['observer', 'notes', 'length of observations', 'date', 'cage #',
       'winner', 'loser', 'action', '1', '2', '3', '4', 'session_number'],
      dtype='object')

    If left blank, the column name will assumed to be "action"
    
Type the actions that will be counted towa

In [36]:
sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].head()

,observer,notes,length of observations,date,cage #,winner,loser,action,1,2,3,4,session_number,session_number_difference
13,Jocelyn,AM,30 min,2022-06-29,6.0,4,2,f,NaN,-1.0,NaN,1.0,1.0,NaN
14,Jocelyn,AM,30 min,2022-06-29,6.0,2,4,f,NaN,1.0,NaN,-1.0,1.0,0.0
16,Jocelyn,AM,30 min,2022-06-29,6.0,2,1,f,-1.0,1.0,NaN,-1.0,1.0,0.0
17,Jocelyn,AM,30 min,2022-06-29,6.0,2,1,c,-1.0,1.0,NaN,-1.0,1.0,0.0
19,Jocelyn,AM,30 min,2022-06-29,6.0,2,1,f,-1.0,1.0,NaN,-1.0,1.0,0.0


In [37]:
sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].tail()

,observer,notes,length of observations,date,cage #,winner,loser,action,1,2,3,4,session_number,session_number_difference
587,Catalina,"while being food restricted, before being fed",30 min,2022-08-09,6.0,1,4,f,1.0,-1.0,-1.0,-1.0,16.0,0.0
588,Catalina,"while being food restricted, before being fed",30 min,2022-08-09,6.0,1,4,f,1.0,-1.0,-1.0,-1.0,16.0,0.0
589,Catalina,"while being food restricted, before being fed",30 min,2022-08-09,6.0,1,2,c,1.0,1.0,-1.0,-1.0,16.0,0.0
590,Catalina,"while being food restricted, before being fed",30 min,2022-08-09,6.0,1,2,c,1.0,1.0,-1.0,-1.0,16.0,0.0
591,Catalina,"while being food restricted, before being fed",30 min,2022-08-09,6.0,1,4,f,1.0,-1.0,-1.0,-1.0,16.0,0.0


## Calculating ELO Score

- Example calculation

In [38]:
eloscore.calculate_elo_score(subject_elo_score=1000, agent_elo_score=2000)

1020

In [39]:
eloscore.update_elo_score(winner_id="A", loser_id="B")

defaultdict(<function eloscore.eloscore.update_elo_score.<locals>.<lambda>()>,
            {'A': 1010.0, 'B': 990.0})

## Get the Elo score for all the events

- Function that creates a dictionary that has the original/updated elo score for each event

In [41]:
for key, value in sheet_name_to_header_row.items():
    
    sheet_name_to_header_row[key]["winner_column"] = input("""Type the name of the column of the subject that has won the interaction.
    i.e. "winner"

    Note: If left blank, the "winner" will be used as the column
    """.format(sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].columns))

    if not sheet_name_to_header_row[key]["winner_column"]:
        sheet_name_to_header_row[key]["winner_column"] = "winner"
    
    sheet_name_to_header_row[key]["loser_column"] = input("""Type the name of the column of the subject that has won the interaction.
    i.e. "loser"

    Note: If left blank, the "loser" will be used as the column
    """.format(sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].columns))

    if not sheet_name_to_header_row[key]["loser_column"]:
        sheet_name_to_header_row[key]["loser_column"] = "loser"
    
    sheet_name_to_header_row[key]["columns_to_keep_string"] = input("""Type all the columns that will be copied from the original dataframe to the ELO Score calculated dataframe. 

    All the available actions: {}
    Each column must be put in quotes and seperated by a comma(,). i.e. 'length of observations', 'date', 'cage #'

    NOTE: If left blank, all the columns will be kept
    """.format(sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].columns))

    
    # Making a list out of the string of inputted sheet names
    if sheet_name_to_header_row[key]["columns_to_keep_string"]:
        sheet_name_to_header_row[key]["columns_to_keep_string"] = "[" + sheet_name_to_header_row[key]["columns_to_keep_string"] + "]"
        sheet_name_to_header_row[key]["columns_to_keep_list"] = ast.literal_eval(sheet_name_to_header_row[key]["columns_to_keep_string"])
    # Using all the sheet names if no sheet name is specified
    else:
        sheet_name_to_header_row[key]["columns_to_keep_list"] =  sorted(list(set(sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].columns) - set([sheet_name_to_header_row[key]["winner_column"]]) - set([sheet_name_to_header_row[key]["loser_column"]])))

    
    sheet_name_to_header_row[key]["index_to_elo_score_and_meta_data"] = eloscore.iterate_elo_score_calculation_for_data_frame(data_frame=sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"], winner_column=sheet_name_to_header_row[key]["winner_column"], loser_column=sheet_name_to_header_row[key]["loser_column"], additional_columns=sheet_name_to_header_row[key]["columns_to_keep_list"] + ["session_number_difference"])
    
    sheet_name_to_header_row[key]["elo_score_df"] = pd.DataFrame.from_dict(sheet_name_to_header_row[key]["index_to_elo_score_and_meta_data"], orient="index")

Type the name of the column of the subject that has won the interaction.
    i.e. "winner"

    Note: If left blank, the "winner" will be used as the column
    
Type the name of the column of the subject that has won the interaction.
    i.e. "loser"

    Note: If left blank, the "loser" will be used as the column
    
Type all the columns that will be copied from the original dataframe to the ELO Score calculated dataframe. 

    All the available actions: Index(['observer', 'notes', 'length of observations', 'date', 'cage #',
       'winner', 'loser', 'action', '1', '2', '3', '4', 'session_number',
       'session_number_difference'],
      dtype='object')
    Each column must be put in quotes and seperated by a comma(,). i.e. 'length of observations', 'date', 'cage #'

    NOTE: If left blank, all the columns will be kept
    
Type the name of the column of the subject that has won the interaction.
    i.e. "winner"

    Note: If left blank, the "winner" will be used as the column


In [42]:
sheet_name_to_header_row[key]["elo_score_df"].head()

,total_match_number,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss,1,2,3,4,action,cage #,date,length of observations,notes,observer,session_number,session_number_difference
0,1,4,2,1000.0,1010.0,1,NaN,-1.0,NaN,1.0,f,6.0,2022-06-29,30 min,AM,Jocelyn,1.0,NaN
1,1,2,2,1000.0,990.0,1,NaN,-1.0,NaN,1.0,f,6.0,2022-06-29,30 min,AM,Jocelyn,1.0,NaN
2,2,2,4,990.0,1000.6,1,NaN,1.0,NaN,-1.0,f,6.0,2022-06-29,30 min,AM,Jocelyn,1.0,0.0
3,2,4,4,1010.0,999.4,1,NaN,1.0,NaN,-1.0,f,6.0,2022-06-29,30 min,AM,Jocelyn,1.0,0.0
4,3,2,1,1000.6,1010.6,1,-1.0,1.0,NaN,-1.0,f,6.0,2022-06-29,30 min,AM,Jocelyn,1.0,0.0


In [43]:
sheet_name_to_header_row[key]["elo_score_df"].tail()

,total_match_number,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss,1,2,3,4,action,cage #,date,length of observations,notes,observer,session_number,session_number_difference
531,266,2,2,780.7,780.2,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0
532,267,1,2,1422.3,1422.8,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0
533,267,2,2,780.2,779.7,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0
534,268,1,4,1422.8,1425.3,1,1.0,-1.0,-1.0,-1.0,f,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0
535,268,4,4,1082.9,1080.4,1,1.0,-1.0,-1.0,-1.0,f,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0


In [44]:
sheet_name_to_header_row[key]["elo_score_df"].groupby("subject_id").count()

,total_match_number,agent_id,original_elo_score,updated_elo_score,win_draw_loss,1,2,3,4,action,cage #,date,length of observations,notes,observer,session_number,session_number_difference
subject_id,,,,,,,,,,,,,,,,,
1,167,167,167,167,167,167,167,163,167,167,167,167,167,167,167,167,167
2,112,112,112,112,112,110,112,106,112,112,112,112,112,112,112,112,111
3,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105
4,152,152,152,152,152,150,152,150,152,152,152,152,152,152,152,152,151


## Getting the cage number

In [45]:
for key, value in sheet_name_to_header_row.items():
    sheet_name_to_header_row[key]["cage_column"] = input("""Type the name of the column of the subject that has won the interaction.
    i.e. "cage"

    Note: If left blank, the column with "cage" will be used as the column
    """.format(sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].columns))

    if not sheet_name_to_header_row[key]["cage_column"]:
        sheet_name_to_header_row[key]["cage_column"] = [col for col in sheet_name_to_header_row[key]["processed_behavior_recording_dataframe"].columns if "cage" in col.lower()][0]
    
    sheet_name_to_header_row[key]["cage_string_to_number"] = defaultdict(dict)
    unique_cage_numbers = sheet_name_to_header_row[key]["elo_score_df"][sheet_name_to_header_row[key]["cage_column"]].unique()
    for cage_string in unique_cage_numbers:
        sheet_name_to_header_row[key]["cage_string_to_number"][cage_string] = re.findall(r'\d+', str(cage_string))[0]
        print(sheet_name_to_header_row[key]["cage_string_to_number"])
    
    sheet_name_to_header_row[key]["elo_score_df"]["processed_cage_number"] = sheet_name_to_header_row[key]["elo_score_df"][sheet_name_to_header_row[key]["cage_column"]].map(sheet_name_to_header_row[key]["cage_string_to_number"])
    

Type the name of the column of the subject that has won the interaction.
    i.e. "cage"

    Note: If left blank, the column with "cage" will be used as the column
    
defaultdict(<class 'dict'>, {1.0: '1'})
Type the name of the column of the subject that has won the interaction.
    i.e. "cage"

    Note: If left blank, the column with "cage" will be used as the column
    
defaultdict(<class 'dict'>, {2: '2'})
defaultdict(<class 'dict'>, {2: '2', 'cage 2': '2'})
Type the name of the column of the subject that has won the interaction.
    i.e. "cage"

    Note: If left blank, the column with "cage" will be used as the column
    
defaultdict(<class 'dict'>, {3: '3'})
defaultdict(<class 'dict'>, {3: '3', 'cage 3': '3'})
Type the name of the column of the subject that has won the interaction.
    i.e. "cage"

    Note: If left blank, the column with "cage" will be used as the column
    
defaultdict(<class 'dict'>, {4: '4'})
Type the name of the column of the subject that has won the 

In [46]:
sheet_name_to_header_row[key]["elo_score_df"].head()

,total_match_number,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss,1,2,3,4,action,cage #,date,length of observations,notes,observer,session_number,session_number_difference,processed_cage_number
0,1,4,2,1000.0,1010.0,1,NaN,-1.0,NaN,1.0,f,6.0,2022-06-29,30 min,AM,Jocelyn,1.0,NaN,6
1,1,2,2,1000.0,990.0,1,NaN,-1.0,NaN,1.0,f,6.0,2022-06-29,30 min,AM,Jocelyn,1.0,NaN,6
2,2,2,4,990.0,1000.6,1,NaN,1.0,NaN,-1.0,f,6.0,2022-06-29,30 min,AM,Jocelyn,1.0,0.0,6
3,2,4,4,1010.0,999.4,1,NaN,1.0,NaN,-1.0,f,6.0,2022-06-29,30 min,AM,Jocelyn,1.0,0.0,6
4,3,2,1,1000.6,1010.6,1,-1.0,1.0,NaN,-1.0,f,6.0,2022-06-29,30 min,AM,Jocelyn,1.0,0.0,6


In [47]:
sheet_name_to_header_row[key]["elo_score_df"].tail()



,total_match_number,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss,1,2,3,4,action,cage #,date,length of observations,notes,observer,session_number,session_number_difference,processed_cage_number
531,266,2,2,780.7,780.2,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6
532,267,1,2,1422.3,1422.8,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6
533,267,2,2,780.2,779.7,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6
534,268,1,4,1422.8,1425.3,1,1.0,-1.0,-1.0,-1.0,f,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6
535,268,4,4,1082.9,1080.4,1,1.0,-1.0,-1.0,-1.0,f,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6


## Saving the Elo Score Dataframes

- Create folders to save processed data

In [48]:
for key, value in sheet_name_to_header_row.items():
    sheet_name_to_header_row[key]["elo_score_df"]["experiment_type"] = "home_cage_observation"

In [49]:
all_sheet_elo_scord_df_list = []
for key, value in sheet_name_to_header_row.items():    
    all_sheet_elo_scord_df_list.append(sheet_name_to_header_row[key]["elo_score_df"])
    
all_sheet_elo_scord_df_combined = pd.concat(all_sheet_elo_scord_df_list)

In [50]:
all_sheet_elo_scord_df_combined.head()

,total_match_number,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss,1,2,3,4,action,cage #,date,length of observations,notes,observer,session_number,session_number_difference,processed_cage_number,experiment_type
0,1,1.0,4.0,1000.0,1010.0,1,1.0,NaN,NaN,-1.0,c,1.0,2022-07-02,30 min,lots of self grooming and very little else,Nancy,6.0,NaN,1,home_cage_observation
1,1,4.0,4.0,1000.0,990.0,1,1.0,NaN,NaN,-1.0,c,1.0,2022-07-02,30 min,lots of self grooming and very little else,Nancy,6.0,NaN,1,home_cage_observation
2,2,1.0,3.0,1010.0,1019.7,1,1.0,NaN,-1.0,-1.0,f,1.0,2022-07-04,30 min,they were trying to take bedding from each other,erika,9.0,3.0,1,home_cage_observation
3,2,3.0,3.0,1000.0,990.3,1,1.0,NaN,-1.0,-1.0,f,1.0,2022-07-04,30 min,they were trying to take bedding from each other,erika,9.0,3.0,1,home_cage_observation
4,3,4.0,3.0,990.0,1000.0,1,1.0,NaN,-1.0,1.0,f,1.0,2022-07-04,30 min,trying to take bedding from the other,erika,9.0,0.0,1,home_cage_observation


In [51]:
all_sheet_elo_scord_df_combined.tail()

,total_match_number,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss,1,2,3,4,action,cage #,date,length of observations,notes,observer,session_number,session_number_difference,processed_cage_number,experiment_type
531,266,2.0,2.0,780.7,780.2,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6,home_cage_observation
532,267,1.0,2.0,1422.3,1422.8,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6,home_cage_observation
533,267,2.0,2.0,780.2,779.7,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6,home_cage_observation
534,268,1.0,4.0,1422.8,1425.3,1,1.0,-1.0,-1.0,-1.0,f,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6,home_cage_observation
535,268,4.0,4.0,1082.9,1080.4,1,1.0,-1.0,-1.0,-1.0,f,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6,home_cage_observation


In [52]:
all_sheet_elo_scord_df_combined.groupby(['subject_id','processed_cage_number']).count()

total_match_number  agent_id  \
subject_id processed_cage_number                                 
1.0        1                                      19        19   
           2                                      35        35   
           3                                      84        84   
           4                                      73        73   
           5                                      86        86   
           6                                     167       167   
2.0        1                                      35        35   
           2                                      49        49   
           3                                      34        34   
           4                                     154       154   
           5                                      92        92   
           6                                     112       112   
3.0        1                                      16        16   
           2                                      98        98   
           3                                      48        48   
           4                                      67        67   
           5                                     133       133   
           6                                     105       105   
4.0        1                                      10        10   
           2                                      28        28   
           3                                      28        28   
           4                                      52        52   
           5                                     161       161   
           6                                     152       152   

                                  original_elo_score  updated_elo_score  \
subject_id processed_cage_number                                          
1.0        1                                      19                 19   
           2                                      35                 35   
           3                                      84                 84   
           4                                      73                 73   
           5                                      86                 86   
           6                                     167                167   
2.0        1                                      35                 35   
           2                                      49                 49   
           3                                      34                 34   
           4                                     154                154   
           5                                      92                 92   
           6                                     112                112   
3.0        1                                      16                 16   
           2                                      98                 98   
           3                                      48                 48   
           4                                      67                 67   
           5                                     133                133   
           6                                     105                105   
4.0        1                                      10                 10   
           2                                      28                 28   
           3                                      28                 28   
           4                                      52                 52   
           5                                     161                161   
           6                                     152                152   

                                  win_draw_loss    1    2    3    4  action  \
subject_id processed_cage_number                                              
1.0        1                                 19   19   16   18   19      19   
           2                                 35   35   35   35   35      35   
           3                                 84   

In [53]:
# importing product
from itertools import product


In [54]:
# apply product method
all_id_and_cage_combination = list(product(all_sheet_elo_scord_df_combined['subject_id'].unique(), all_sheet_elo_scord_df_combined['processed_cage_number'].unique()))

In [55]:
all_sheet_elo_scord_df_groupby = all_sheet_elo_scord_df_combined.groupby(['subject_id','processed_cage_number']).size().reset_index()

all_sheet_elo_scord_df_groupby = all_sheet_elo_scord_df_groupby[all_sheet_elo_scord_df_groupby[0] >= 1]

index_to_subject_id_and_processed_cage_number = defaultdict(dict)
for index, row in all_sheet_elo_scord_df_groupby.iterrows():
    index_to_subject_id_and_processed_cage_number[index]['subject_id'] = row['subject_id']
    index_to_subject_id_and_processed_cage_number[index]['processed_cage_number'] = row['processed_cage_number']


In [56]:
index_to_subject_id_and_processed_cage_number

defaultdict(dict,
            {0: {'subject_id': 1.0, 'processed_cage_number': '1'},
             1: {'subject_id': 1.0, 'processed_cage_number': '2'},
             2: {'subject_id': 1.0, 'processed_cage_number': '3'},
             3: {'subject_id': 1.0, 'processed_cage_number': '4'},
             4: {'subject_id': 1.0, 'processed_cage_number': '5'},
             5: {'subject_id': 1.0, 'processed_cage_number': '6'},
             6: {'subject_id': 2.0, 'processed_cage_number': '1'},
             7: {'subject_id': 2.0, 'processed_cage_number': '2'},
             8: {'subject_id': 2.0, 'processed_cage_number': '3'},
             9: {'subject_id': 2.0, 'processed_cage_number': '4'},
             10: {'subject_id': 2.0, 'processed_cage_number': '5'},
             11: {'subject_id': 2.0, 'processed_cage_number': '6'},
             12: {'subject_id': 3.0, 'processed_cage_number': '1'},
             13: {'subject_id': 3.0, 'processed_cage_number': '2'},
             14: {'subject_id': 3.0, 'pr

In [57]:
all_sheet_elo_scord_df_combined["processed_cage_number"].unique()

array(['1', '2', '3', '4', '5', '6'], dtype=object)

In [60]:
id_to_final_elo_score_dict = defaultdict(dict)
for index, (key, value) in enumerate(index_to_subject_id_and_processed_cage_number.items()):

    per_subject_df = all_sheet_elo_scord_df_combined[(all_sheet_elo_scord_df_combined["subject_id"] == value["subject_id"]) & (all_sheet_elo_scord_df_combined["processed_cage_number"] == value["processed_cage_number"])]
   
    id_to_final_elo_score_dict[index]["processed_cage_number"] = value["processed_cage_number"]
    id_to_final_elo_score_dict[index]["subject_id"] = value["subject_id"]
    id_to_final_elo_score_dict[index]["final_elo_score"] = per_subject_df.iloc[-1]["updated_elo_score"]

In [61]:
id_to_final_elo_score_dict

defaultdict(dict,
            {0: {'processed_cage_number': '1',
              'subject_id': 1.0,
              'final_elo_score': 904.2},
             1: {'processed_cage_number': '2',
              'subject_id': 1.0,
              'final_elo_score': 881.2},
             2: {'processed_cage_number': '3',
              'subject_id': 1.0,
              'final_elo_score': 1281.3},
             3: {'processed_cage_number': '4',
              'subject_id': 1.0,
              'final_elo_score': 1079.9},
             4: {'processed_cage_number': '5',
              'subject_id': 1.0,
              'final_elo_score': 939.5},
             5: {'processed_cage_number': '6',
              'subject_id': 1.0,
              'final_elo_score': 1425.3},
             6: {'processed_cage_number': '1',
              'subject_id': 2.0,
              'final_elo_score': 1194.5},
             7: {'processed_cage_number': '2',
              'subject_id': 2.0,
              'final_elo_score': 894.6},
          

In [62]:
id_to_final_elo_score_df = pd.DataFrame.from_dict(id_to_final_elo_score_dict, orient="index")
id_to_final_elo_score_df["experiment_type"] = "home_cage_observation"

In [63]:
id_to_final_elo_score_df

,processed_cage_number,subject_id,final_elo_score,experiment_type
0,1,1.0,904.2,home_cage_observation
1,2,1.0,881.2,home_cage_observation
2,3,1.0,1281.3,home_cage_observation
3,4,1.0,1079.9,home_cage_observation
4,5,1.0,939.5,home_cage_observation
5,6,1.0,1425.3,home_cage_observation
6,1,2.0,1194.5,home_cage_observation
7,2,2.0,894.6,home_cage_observation
8,3,2.0,898.4,home_cage_observation
9,4,2.0,1348.9,home_cage_observation


# Making plots for all sheets

- Getting the dates the files were being recorded to use for the file name

In [64]:
all_sheet_elo_scord_df_combined

,total_match_number,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss,1,2,3,4,action,cage #,date,length of observations,notes,observer,session_number,session_number_difference,processed_cage_number,experiment_type
0,1,1.0,4.0,1000.0,1010.0,1,1.0,NaN,NaN,-1.0,c,1.0,2022-07-02,30 min,lots of self grooming and very little else,Nancy,6.0,NaN,1,home_cage_observation
1,1,4.0,4.0,1000.0,990.0,1,1.0,NaN,NaN,-1.0,c,1.0,2022-07-02,30 min,lots of self grooming and very little else,Nancy,6.0,NaN,1,home_cage_observation
2,2,1.0,3.0,1010.0,1019.7,1,1.0,NaN,-1.0,-1.0,f,1.0,2022-07-04,30 min,they were trying to take bedding from each other,erika,9.0,3.0,1,home_cage_observation
3,2,3.0,3.0,1000.0,990.3,1,1.0,NaN,-1.0,-1.0,f,1.0,2022-07-04,30 min,they were trying to take bedding from each other,erika,9.0,3.0,1,home_cage_observation
4,3,4.0,3.0,990.0,1000.0,1,1.0,NaN,-1.0,1.0,f,1.0,2022-07-04,30 min,trying to take bedding from the other,erika,9.0,0.0,1,home_cage_observation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,266,2.0,2.0,780.7,780.2,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6,home_cage_observation
532,267,1.0,2.0,1422.3,1422.8,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6,home_cage_observation
533,267,2.0,2.0,780.2,779.7,1,1.0,1.0,-1.0,-1.0,c,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6,home_cage_observation
534,268,1.0,4.0,1422.8,1425.3,1,1.0,-1.0,-1.0,-1.0,f,6.0,2022-08-09,30 min,"while being food restricted, before being fed",Catalina,16.0,0.0,6,home_cage_observation


In [ ]:
raise ValueError()

In [66]:
all_earlist_dates = []
all_latest_dates = []
for key, value in sheet_name_to_header_row.items():
    print(key)
    all_earlist_dates.append(sheet_name_to_header_row[key]["elo_score_df"][sheet_name_to_header_row[key]["session_divider_column"]].min())
    all_earlist_dates.append(sheet_name_to_header_row[key]["elo_score_df"][sheet_name_to_header_row[key]["session_divider_column"]].max())

CAGE1
CAGE2
CAGE3
CAGE4
CAGE5
CAGE6
23


KeyError: 'elo_score_df'

In [68]:
sheet_name_to_header_row.keys()

dict_keys(['CAGE1', 'CAGE2', 'CAGE3', 'CAGE4', 'CAGE5', 'CAGE6', 23])

In [ ]:
earliest_date = str(sheet_to_data_frame[sheets[1]]["recording_data_frame"]["date"].min()).split()[0]

In [ ]:
latest_date = str(sheet_to_data_frame[sheets[1]]["recording_data_frame"]["date"].max()).split()[0]

In [ ]:
earliest_date

In [ ]:
latest_date

- Getting the cage numbers

In [ ]:
all_cages = "_".join(sorted([sheet.lower().strip("cage").strip() for sheet in sheet_to_data_frame.keys()]))

In [ ]:
all_cages

In [ ]:
output_directory = "./proc/plots/home_cage_elo_score/cage_{}_date_{}_{}".format(all_cages, earliest_date, latest_date)

In [ ]:
output_directory

In [ ]:
os.makedirs(output_directory, exist_ok=True)

In [ ]:
for key, value in sheet_to_data_frame.items():
    plt.rcParams["figure.figsize"] = (13.5,7.5)
    print(key)
    elo_score_data_frame = value["elo_score_data_frame"]
    fig, ax = plt.subplots()

    # Drawing vertical lines that represent when each session begins
    for index, row in elo_score_data_frame.dropna(subset=["session_number_plotting"]).iterrows():
        # Offsetting by 0.5 to avoid drawing the line on the dot
        plt.vlines(x=[row["total_match_number"] - 0.5], ymin=600, ymax=1500, colors='black', linestyle='dashed')

    # Drawing a line for each subject
    for subject in sorted(elo_score_data_frame["subject_id"].astype(str).unique()):
        # Getting all the rows with the current subject
        subject_df = elo_score_data_frame[elo_score_data_frame["subject_id"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        plt.plot(subject_df["total_match_number"], subject_df["updated_elo_score"], '-o', label=subject)

    # Labeling the X/Y Axis and the title
    ax.set_xlabel("Trial Number")
    ax.set_ylabel("Elo Score")
    ax.set_title("Elo Score for Home Cage Observation: {}".format(key))
    # To show the legend
    ax.legend()
    plt.xticks(rotation=90)
    plt.ylim(600, 1500) 
    
    
    file_name = "home_cage_elo_score_{}_date_{}_{}.png".format(key, earliest_date, latest_date)
    plt.savefig(os.path.join(output_directory, file_name))
    
    plt.show()

# Saving the Dataframes

In [ ]:
output_directory = "./proc/elo_score_spread_sheets/home_cage_observation_elo_score/cage_{}_date_{}_{}".format(all_cages, earliest_date, latest_date)

In [ ]:
output_directory

In [ ]:
os.makedirs(output_directory, exist_ok=True)

In [ ]:
file_name = "home_cage_observation_elo_score_history_cages_{}_date_{}_{}.csv".format(all_cages, earliest_date, latest_date)
all_sheet_elo_scord_df_combined.to_csv(os.path.join(output_directory, file_name))

In [ ]:
file_name = "home_cage_observation_final_elo_score_cages_{}_date_{}_{}.csv".format(all_cages, earliest_date, latest_date)
id_to_final_elo_score_df.to_csv(os.path.join(output_directory, file_name))

# For All sheets

# Example using One Sheet

In [ ]:
home_cage_observations_df = pd.read_excel(raw_data_file_path, sheet_name=inputted_sheet_names_list[0], header=header_row)

In [ ]:
home_cage_observations_df.head()

## Standarizing the Column

In [ ]:
original_column_name_to_standarized = defaultdict(dict)
for col in home_cage_observations_df.columns:
    original_column_name_to_standarized[col] = str(col).lower().strip()

In [ ]:
original_column_name_to_standarized

In [ ]:
home_cage_observations_df = home_cage_observations_df.rename(columns=original_column_name_to_standarized)

In [ ]:
home_cage_observations_df

In [ ]:
new_session_indexes

# Plotting the Elo Score by match number

In [ ]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

In [ ]:
fig, ax = plt.subplots()

# Drawing vertical lines that represent when each session begins
for index, row in home_cage_elo_score_df[home_cage_elo_score_df['session_number_difference'] != 0.0].iterrows():
    # Offsetting by 0.5 to avoid drawing the line on the dot
    plt.vlines(x=[row["total_match_number"] - 0.5], ymin=700, ymax=1300, colors='black', linestyle='dashed')

# Drawing a line for each subject
for subject in sorted(home_cage_elo_score_df["subject_id"].unique()):
    # Getting all the rows with the current subject
    subject_df = home_cage_elo_score_df[home_cage_elo_score_df["subject_id"] == subject]
    # Making the dates into days after the first session by subtracting all the dates by the first date
    plt.plot(subject_df["total_match_number"], subject_df["updated_elo_score"], '-o', label=subject)

# Labeling the X/Y Axis and the title
ax.set_xlabel("Trial Number")
ax.set_ylabel("Elo Score")
# ax.set_title("Elo Score for Home Cage Observation: Cage {}".format(cage))
# To show the legend
ax.legend()
plt.xticks(rotation=90)
plt.ylim(700, 1300) 
plt.show()



# ELO Score for all sheets (final)

# Calculate the Elo Score for all sheets

In [ ]:
xls = pd.ExcelFile(raw_data_file_path)
sheets = xls.sheet_names

In [ ]:
sheets

In [ ]:
sheet_to_data_frame = defaultdict(dict)
for sheet in sheets[:]:
    try:
        # Seeing if "Unnamed" is in any of the cells for the 0th row
        if "Unnamed" not in "".join([str(col) for col in pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0).columns]):
            header_row = 0
        # Seeing if "Unnamed" is in any of the cells for the 1st row
        elif "Unnamed" not in "".join([str(col) for col in pd.read_excel(raw_data_file_path, sheet_name=sheet, header=1).columns]):
            header_row = 1
        else:
            header_row = 0
            warnings.warn("""Both the 0th and the 1st row have cells with blank values. This means that that these rows can not be used as headers. Please change the row number to the header""")

        
        
        
        sheet_to_data_frame[sheet]["recording_data_frame"] = pd.read_excel(raw_data_file_path, sheet_name=sheet, header=header_row).copy()

        original_column_name_to_standarized = defaultdict(dict)
        for col in sheet_to_data_frame[sheet]["recording_data_frame"].columns:
            original_column_name_to_standarized[col] = str(col).lower().strip()

        sheet_to_data_frame[sheet]["recording_data_frame"] = sheet_to_data_frame[sheet]["recording_data_frame"].rename(columns=original_column_name_to_standarized)

        sheet_to_data_frame[sheet]["recording_data_frame"] = sheet_to_data_frame[sheet]["recording_data_frame"].dropna(subset=["winner"])

        # Adding the session numbers as a column
        session_indexes = sheet_to_data_frame[sheet]["recording_data_frame"].dropna(axis=0, subset="date").index
        sheet_to_data_frame[sheet]["recording_data_frame"] = eloscore.add_session_number_column(sheet_to_data_frame[sheet]["recording_data_frame"], session_indexes)
        sheet_to_data_frame[sheet]["recording_data_frame"] = sheet_to_data_frame[sheet]["recording_data_frame"].fillna(method='ffill')
        sheet_to_data_frame[sheet]["recording_data_frame"] = eloscore.add_session_number_column(sheet_to_data_frame[sheet]["recording_data_frame"], session_indexes, session_number_column="session_number_plotting")

        sheet_to_data_frame[sheet]["recording_data_frame"] = sheet_to_data_frame[sheet]["recording_data_frame"][sheet_to_data_frame[sheet]["recording_data_frame"]["action"].isin(["f", "c"])]

        # Splitting all the rows with commas in the ID
        sheet_to_data_frame[sheet]["recording_data_frame"]["winner"] = sheet_to_data_frame[sheet]["recording_data_frame"]["winner"].astype(str)
        sheet_to_data_frame[sheet]["recording_data_frame"]["loser"] = sheet_to_data_frame[sheet]["recording_data_frame"]["loser"].astype(str)

        sheet_to_data_frame[sheet]["recording_data_frame"]["winner"] = sheet_to_data_frame[sheet]["recording_data_frame"]["winner"].str.split(",")
        sheet_to_data_frame[sheet]["recording_data_frame"] = sheet_to_data_frame[sheet]["recording_data_frame"].explode("winner").reset_index(drop=True)
        sheet_to_data_frame[sheet]["recording_data_frame"]["loser"] = sheet_to_data_frame[sheet]["recording_data_frame"]["loser"].str.split(",")
        sheet_to_data_frame[sheet]["recording_data_frame"] = sheet_to_data_frame[sheet]["recording_data_frame"].explode("loser").reset_index(drop=True)

        sheet_to_data_frame[sheet]["recording_data_frame"]["winner"] = sheet_to_data_frame[sheet]["recording_data_frame"]["winner"].astype(float).astype(int).astype(str)
        sheet_to_data_frame[sheet]["recording_data_frame"]["loser"] = sheet_to_data_frame[sheet]["recording_data_frame"]["loser"].astype(float).astype(int).astype(str)

        # Calculating the Elo Score    
        index_to_elo_score_and_meta_data = eloscore.iterate_elo_score_calculation_for_data_frame(data_frame=sheet_to_data_frame[sheet]["recording_data_frame"], \
                                                                                                                   winner_column="winner", loser_column="loser", additional_columns=["date", "session_number", "session_number_plotting"])
        sheet_to_data_frame[sheet]["elo_score_data_frame"] = pd.DataFrame.from_dict(index_to_elo_score_and_meta_data, orient="index")
    except:
        print(sheet)

        warnings.warn("Please update the formatting for the sheet with the above name, or continue if this sheet is being ignored")
        sheet_to_data_frame.pop(sheet, None)

In [ ]:
sheet_to_data_frame.keys()